In [1]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from function import Utils
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas.api.types import is_numeric_dtype
import numpy as np
import h2o
from h2o.automl import H2OAutoML
from IPython.display import display
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("hmw/hospital_billing.csv",index_col=0)
df.head()

,case_id,activity,lifecycle,resource,timestamp,actorange,actred,blocked,casetype,closecode,...,iscancelled,isclosed,msgcode,msgcount,msgtype,speciality,state,version,activity_instance_id,.order
1,A,NEW,complete,ResA,2012-12-16 19:33:10,NaN,NaN,False,A,NaN,...,False,True,NaN,NaN,NaN,A,In progress,NaN,1,1
2,A,FIN,complete,NaN,2013-12-15 19:00:37,NaN,NaN,NaN,NaN,A,...,NaN,NaN,NaN,NaN,NaN,NaN,Closed,NaN,2,2
3,A,RELEASE,complete,NaN,2013-12-16 03:53:38,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Released,NaN,3,3
4,A,CODE OK,complete,NaN,2013-12-17 12:56:29,False,False,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,A,4,4
5,A,BILLED,complete,ResB,2013-12-19 03:44:31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Billed,NaN,5,5


In [3]:
Utils.persantage_nan(df)

case_id                  0.002002
activity                 0.000000
lifecycle                0.000000
resource                47.198254
timestamp                0.000000
actorange               85.724010
actred                  85.693980
blocked                 79.980381
casetype                79.642049
closecode               84.793097
diagnosis               80.344738
flaga                   79.980381
flagb                   79.980381
flagc                   85.693980
flagd                   79.966367
iscancelled             78.014454
isclosed                79.780185
msgcode                 99.725731
msgcount                85.591880
msgtype                 99.725731
speciality              79.980381
state                   15.721407
version                 85.021321
activity_instance_id     0.000000
.order                   0.000000
dtype: float64

In [4]:
input = {
    "case_id" : "caseid",
    "activity" : "activity" ,
    "resource" : "resource" ,
    "timestamp": "ts",
    "state" : "y"  #Needs to be 1 for each Case
}

In [5]:
# Change columns name
df.rename(columns=input,inplace = True)

In [6]:

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

df["dt"] = Utils.dt(df) # Thisneed to be improved
df.drop("ts",axis=1,inplace = True)

In [7]:
df = Utils.prod_nan_with_treshold(df,treshold=75)

In [8]:
df,model = Utils.predict_null_value(colum="resource",df=df)

In [9]:
df,model = Utils.predict_null_value(colum="y",df=df)

In [11]:
df = Utils.drop_id_columns(df, drop_id_threshold=0.9)

Dropping column activity_instance_id with value: 1.0
Dropping column .order with value: 1.0


In [12]:
df.head()

,caseid,activity,lifecycle,resource,y,dt,resource_was_null,y_was_null
1,A,NEW,complete,ResA,In progress,363 days 23:27:27,False,False
2,A,FIN,complete,ResA,Closed,0 days 08:53:01,True,False
3,A,RELEASE,complete,ResA,Released,1 days 09:02:51,True,False
4,A,CODE OK,complete,ResA,Released,1 days 14:48:02,True,True
5,A,BILLED,complete,ResB,Billed,1 days 14:48:02,False,False


In [13]:
df_grouped = df.groupby(["caseid"]).agg(list).reset_index()
df_grouped.head()

,caseid,activity,lifecycle,resource,y,dt,resource_was_null,y_was_null
0,A,"[NEW, FIN, RELEASE, CODE OK, BILLED]","[complete, complete, complete, complete, compl...","[ResA, ResA, ResA, ResA, ResB]","[In progress, Closed, Released, Released, Billed]","[363 days 23:27:27, 0 days 08:53:01, 1 days 09...","[False, True, True, True, False]","[False, False, False, True, False]"
1,AA,"[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...","[complete, complete, complete, complete, compl...","[ResHA, ResOA, ResA, ResA, ResA, ResB]","[In progress, In progress, Closed, Released, I...","[0 days 00:00:41, 50 days 12:15:34, 0 days 01:...","[False, True, False, True, True, False]","[False, False, False, False, True, False]"
2,AAA,[NEW],[complete],[ResJA],[In progress],[19 days 12:36:49],[False],[False]
3,AAB,[NEW],[complete],[ResJA],[In progress],[153 days 12:20:19],[False],[False]
4,AAC,"[NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...","[complete, complete, complete, complete, compl...","[ResVG, ResJA, ResA, ResA, ResJA, ResB]","[In progress, In progress, Closed, Released, I...","[0 days 00:00:18, 7 days 08:13:48, 0 days 06:5...","[False, True, False, True, True, False]","[False, False, False, False, True, False]"


In [14]:
df_grouped = Utils.reduce_list_columns(df_grouped)

In [15]:
df_grouped_agg = Utils.aggregation_encoding(df_grouped,df)